## Making our Bot Interactive

So far, we have a simple newsbot that posts top news headlines to our Slack channel. This is a nice start but we'd really like to query our bot on demand to ask for top headlines, news from a particual site or even news about a particular topic. I won't go as far as calling it "conversational" but I'd like our bot to be more interactive. To be able to talk back-and-forth with our Slack bot, we have two options. We can use:
* [Slack Events](https://api.slack.com/events-api), or
* [Real Time Messaging](https://slack.dev/python-slackclient/real_time_messaging.html)

Today, we're going to use Slack Events, which allows us to register for certain events we'd like to hear about (a Slack user sends our bot a message, or a someone adds an emoji to a Slack message, for example). When one of the events occurs, Slack will send us a message, letting us know about it. 


[TODO] note about building a web service/server and Slack sending us HTTP requests

We'll be using flask.


In [ ]:
!pip install flask

In [ ]:
from flask import Flask

# This `app` represents our Flask app
app = Flask(__name__)

# we can create "routes" which tell our app where to "route" incoming requests
@app.route("/")
def hello():
    return "Hello there!"

# this is a simple "/api" endpoint
@app.route("/api")
def our_api():
    return "If we had an API, it could go here!"

# start our Flask app and have it run on port 5000
app.run(port=5000)

**When you run the cell above, you should see:**

```
* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```


What is `127.0.0.1`? And, what's the `5000`?

`127.0.0.1`, also called "[localhost](https://en.wikipedia.org/wiki/Localhost)", is the IP address of our computer.


![There's no place like 127.0.0.1](https://bookofjoe.typepad.com/photos/uncategorized/2008/12/07/1htrd.jpg "There's no place like 127.0.0.1")

As for the `5000` - that is the port number assigned to our Flask application. We communicate with our app by connecting to it via the IP address (127.0.0.1) and port number.

Well-known ports include 80 for ordinary HTTP and 443 for HTTPS traffic. A port can be any number from 0 to 65535. The low numbers (below 1024) are reserved for your computer and are managed by an administrator. Ports from [1024 through 49151 are "registered" by IANA](https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.xhtml), the Internet Assigned Numbers Authority. [Here's a nice list of well-known and registered ports](https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers). (Look for 8888 and you'll see it's associated with our notebooks! Oh and look at this browser window's navigation bar to see 8888 in the address of this notebook.) Now, numbers beyond these ranges are a zoo... they are for custom, often temporary communication. They are referred to as "private/dynamic". 


### Creating a connection from Slack to our Bot

Now, we'll be running the bot code on our laptop. But Slack requires bots to run behind a public HTTPS server. To accomplish this easily, we'll need some help from a tool called ngrok which will route information to and from Slack for us.

At a technical level, ngrok is a command-line program that opens a secure tunnel to localhost and exposes that tunnel behind an HTTPS endpoint. ngrok makes it so Slack can talk to your code right away. 

<img src="https://cloud.githubusercontent.com/assets/32463/25376866/940435fa-299d-11e7-9ee3-08d9427417f6.png">

Follow the next three steps to install and run ngrok:

1. [Download](https://ngrok.com/download) the ngrok client for your operating system.
2. Unzip it to a location you can remember.
3. Open up a new terminal, cd into the location, and enter:


on Unix/Mac:

      ./ngrok http 5000

on Windows:

      ngrok.exe http 5000
What you should see is something like this.

<img src="https://github.com/computationaljournalism/columbia2018/raw/master/images/ng.jpg" style="width: 50%; border: #000000 1px outset;"/>

NOTE The important part of the information here is the Forwarding https address - in this case: `https://3c825e6b.ngrok.io`. You will need this in the next step.

### Slack Events

First, we need to enable Slack Events in our app settings and have Slack "verify" us, before we're able to receive events from Slack. To do this, we need to build upon our simple Flask app and create a "route" (or api endpoint) that Slack can call to verify that we are how we say we are. We'll do this using a python library developed by the folks at Slack. Let's install that now:

In [ ]:
!pip install slackeventsapi

We need to get our Slack signing secret, which will be used to verify our bot with Slack. This is similar to an API key but slightly different. 

Go back to our [Slack App settings](https://api.slack.com/apps) and navigate to the `Basic Information` section. Scroll down to `Signing Secret` under the `App Credentials` section and hit the "Show" button. Copy the Signing Secret and paste it below in the `SLACK_SIGNING_SECRET` variable. Once you've done that, run the cell to start up our Slack bot so that Slack can verify that we are legit!

<img src="https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/images/bot_secret.png" width="600">

**Start** the cell below so that our bot (ie web server) is running on port 5000. Then, do the following, to make sure Slack can "verify" our bot:

1. Head back to https://api.slack.com/apps/ and navigate to `Event Subscriptions`. Toggle it on.
2. In the Request URL field, enter your ngrok http URL (from above) and append "/slack/events" to the end of it. It should look like: https://e90229c5.ngrok.io/slack/events
3. Click "Save Changes"

If you see "Request URL Your URL didn't respond with the value of the challenge parameter.", make sure our bot is running below and then hit the Retry button.

We're looking for a Verified ✅ next to the Request URL. Once we have this, we can move on!

In [ ]:
import pprint
from flask import Flask
from slackeventsapi import SlackEventAdapter

# This `app` represents our Flask app
app = Flask(__name__)

# slack signing secret found in: https://api.slack.com/apps
# which is the "Basic Information" tab --> App Credential section --> Show Signing Secret
SLACK_SIGNING_SECRET = ""

slack_events_adapter = SlackEventAdapter(SLACK_SIGNING_SECRET, "/slack/events", app)

# Start the app on port 5000
app.run(port=5000)

[TODO] Add a note about what this ^^^ code is doing

### Enabling @bot Mention Events

Now that our service is verified, we can start to add Events that we'd like to receive from Slack. Since we're building a simple bot that we'd like users to interact with, let's create an Event that allows us to get a message each time we receive an event when our bot `@newsbot` is mentioned in our Slack channel. To do this:

1. Head back to our Slack app settings page and click on `Event Subscriptions` in the left-hand menu
2. Click on `Subscribe to bot events`
3. Click the `Add Bot User Event` button
4. Search for `app_mention` and add that event
5. Click the green `Save Changes` button in the bottom right-hand side of the page
6. Click on the `reinstall your app` link at the top of the screen (in the yellow message). This will redirect you back to your App Installation page. Click `Allow` which will allow your bot to have permissions for the newly create Event(s).

**Now** let's modify our bot code to receive messages whenever our bot is summoned by a user. We can do this by create a function that uses the decorator:
```
@slack_events_adapter.on("app_mention")
def app_mention(event_data):
    print "we got an app mention!"
```

The code below will simply print out the data we receive from Slack when our bot gets an @mention. Run the following code and then send your bot some messages in your Slack channel!

In [ ]:
import pprint

@slack_events_adapter.on("app_mention")
def app_mention(event_data):
    pprint.pprint(event_data)

# Start the app on port 5000
app.run(port=5000)

[TODO]: Add a note here about simply sending back a message when we receivee an @bot event

In [ ]:
import pprint

# let's add back our slack WebClient
# which allows us to post messages back to Slack
slack_client = slack.WebClient(token=SLACK_API_TOKEN)

@slack_events_adapter.on("app_mention")
def app_mention(event_data):
    pprint.pprint(event_data)

    # lets get the channel ID from the incoming message
    # and send "Hi there!" back to that channel
    channel = event_data["event"]["channel"]
    
    response = slack_client.chat_postMessage(channel=channel, text="Hi there!")

# Start the app on port 5000
app.run(port=5000)

### End of Part 2

* Part 1 was about setting up our corona-bot and Slack app setup
* Part 3 will be about building the cornoa-bot functionality (using Slack Events)